In [ ]:
import MDAnalysis as mda
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#-----Uploading trajectory file to MDAnalyisis as a Universe----------

u = mda.Universe("FUS.gsd",lengthunit="A", timeunit="fs")

In [ ]:
#-----Definiton of the 100 Proteins by Particle IDs-------
proteins = {}

for i in range(0,100):
    proteins[i] = u.select_atoms("index "+str(163*i)+":" +str(163*(i+1)-1))

In [ ]:
#---(ALL FRAMES) Main Loop to Determine High and Low Densities and Calculate Interparticle Distances----

num_high_protein = 0
num_low_protein = 0

s_pro_high_dist = np.zeros([163])
s_pro_low_dist = np.zeros([163])
total_pro_high_dist = np.zeros([163])
total_pro_low_dist = np.zeros([163])
avg_all_frame_high_dist = np.zeros([163])
avg_all_frame_low_dist = np.zeros([163])

start_frame = 600
end_frame = 1200
frame = 0

for ts in u.trajectory[start_frame:end_frame] :
  for i in range(0,100):
    bin_coor = proteins[i][80].position[2]
    atom_count = u.select_atoms("prop z>"+str(bin_coor-0.5)+" and prop z<"+str(bin_coor+0.5))
    if np.size(atom_count)>400 :
      for j in range(0,163):
        s_pro_high_dist[j] = math.dist(proteins[i][0].position,proteins[i][j].position)
      total_pro_high_dist = total_pro_high_dist + s_pro_high_dist
      num_high_protein = num_high_protein + 1
    if np.size(atom_count)<70 :
      for j in range(0,163):
        s_pro_low_dist[j] = math.dist(proteins[i][0].position,proteins[i][j].position)
      total_pro_low_dist = total_pro_low_dist + s_pro_low_dist
      num_low_protein = num_low_protein + 1
  avg_s_frame_high_dist = total_pro_high_dist/num_high_protein
  avg_s_frame_low_dist = total_pro_low_dist/num_low_protein
  avg_all_frame_high_dist = avg_all_frame_high_dist + avg_s_frame_high_dist
  avg_all_frame_low_dist = avg_all_frame_low_dist + avg_s_frame_low_dist
  num_high_protein = 0
  num_low_protein = 0
  total_pro_high_dist = 0
  total_pro_low_dist = 0
  frame = frame + 1

total_avg_all_frame_high_dist = avg_all_frame_high_dist/frame
total_avg_all_frame_low_dist = avg_all_frame_low_dist/frame

In [ ]:
#----Saving the Data----

f = open('High_Density_Intermolecular_Distances.txt', 'a')
np.savetxt(f,total_avg_all_frame_high_dist, delimiter=" ")
f.close()

f = open('Low_Density_Intermolecular_Distances.txt', 'a')
np.savetxt(f,total_avg_all_frame_low_dist, delimiter=" ")
f.close()

In [ ]:
#---------Plotting---------
indices = np.linspace(0,162,163)

ax = plt.subplot(111)
ax.plot(indices, total_avg_all_frame_low_dist, 'b', lw=2, label=r"$Low Density$")
ax.plot(indices, total_avg_all_frame_high_dist, 'r', lw=2, label=r"$High Density$")
ax.set_xlabel("|i-j|")
ax.set_ylabel("$R_ij$ (nm)")
ax.legend()
ax.figure.savefig("Intermoleculer_Distances.pdf")
plt.show()

In [ ]:
#---(SINGLE FRAME) Main Loop to Determine High and Low Densities and Calculate Interparticle Distances----


u.trajectory[1166]

num_high_protein = 0
num_low_protein = 0

s_pro_high_dist = np.zeros([163])
s_pro_low_dist = np.zeros([163])
total_pro_high_dist = np.zeros([163])
total_pro_low_dist = np.zeros([163])


for i in range(0,100):
  bin_coor = proteins[i][80].position[2]
  atom_count = u.select_atoms("prop z>"+str(bin_coor-0.5)+" and prop z<"+str(bin_coor+0.5))
  if np.size(atom_count)>400 :
    for j in range(0,163):
      s_pro_high_dist[j] = math.dist(proteins[i][0].position,proteins[i][j].position)
    num_high_protein = num_high_protein + 1
    total_pro_high_dist = total_pro_high_dist + s_pro_high_dist
  if np.size(atom_count)<70 :
    for j in range(0,163):
      s_pro_low_dist[j] = math.dist(proteins[i][0].position,proteins[i][j].position)
    num_low_protein = num_low_protein + 1
    total_pro_low_dist = total_pro_low_dist + s_pro_low_dist

avg_s_frame_high_dist = total_pro_high_dist/num_high_protein
avg_s_frame_low_dist = total_pro_low_dist/num_low_protein